In [1]:
import mlagents
from mlagents_envs.environment import UnityEnvironment as UE
from mlagents_envs.envs.unity_parallel_env import UnityParallelEnv as UPZBE
import numpy as np
import torch
import cv2

In [2]:
env = UE(file_name="DroneFlightv1", seed=1, side_channels=[], no_graphics_monitor=True, no_graphics=True)
env = UPZBE(env)

In [3]:
def relocate_agents():
    return [agent for agent in env.agents]
agents = relocate_agents()

## Test PPO

In [4]:
from PPO_Distillation.DistilledPPOAgent import DistilledPPO
from PPO_Distillation.Trajectories import ExperienceBuffer
from PPO_Distillation.Hyperparameters import HYPERPARAMS as params

c:\Users\rullo\anaconda3\envs\ml_agents\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
brain = DistilledPPO(env.observation_space(agents[0])[1].shape, env.observation_space(agents[0])[2].shape, env.action_space(agents[0]).shape, params['ppo_distilled'])
brain.load_model("./PPO_distilled_checkpoint.pth")

## Test

In [6]:
steps = 0
brain.net.eval()
obs, done, t = env.reset(), [False for _ in env.agents], 0
episode_reward = 0
while not all(done) or t < 10000:
    actions = {}
    agents = relocate_agents()
    for agent in agents:
        if agent not in obs.keys():
            continue
        if isinstance(obs[agent], list):
            actions[agent], _, _ = brain.get_action(obs[agent][1], obs[agent][2])
        else:
            actions[agent], _, _ = brain.get_action(obs[agent]['observation'][1], obs[agent]['observation'][2])
        t += 1

    obs, reward, done, _ = env.step(actions)
    done = [done[agent] for agent in agents if agent in done.keys()]
    tot_reward = [reward[agent] for agent in agents if agent in reward.keys()]
mean_reward = np.mean(tot_reward)
print(f"Mean reward: {mean_reward}")

Mean reward: -1.000100016593933
